# 토이 프로젝트 크롤링을 위한 모듈

In [ ]:
import time
import urllib
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import numpy as np
import pandas as pd

# 키노라이츠 크롤링

In [ ]:
driver = webdriver.Chrome('./chromedriver_win32/chromedriver.exe')
url = 'https://m.kinolights.com/'
driver.get(url)

driver.find_element_by_xpath('/html/body/div/div/div/header/nav/a[2]').click()
time.sleep(3)
driver.find_element_by_css_selector('section.theme-list-section > ul > li:nth-child(6) > a').send_keys('\n')
time.sleep(5)

netflix_name = []
tving_name = []
tot_list = []
def click_contents(c):  
        if c == 'Netflix':
            driver.find_element_by_xpath('//*[@id="triggerMenu"]/div/div[1]/div/div/button[1]/i').click() 
            time.sleep(2)
            driver.find_element_by_xpath('//*[@id="listArea"]/div[1]/button/i').click()
            time.sleep(2)
            driver.find_element_by_xpath('//*[@id="modalContentBody"]/div/div[4]/button').click()
            
            html = BeautifulSoup(driver.page_source, 'html.parser')
            for name1 in html.select('div > div.title'):
                netflix_name.append(name1.text)
                tot_list.append(name1.text)
                
            driver.find_element_by_xpath('//*[@id="triggerMenu"]/div/div[1]/div/div/button[1]/i').click()
                
        elif c == 'Tving':
            driver.find_element_by_xpath('//*[@id="triggerMenu"]/div/div[1]/div/div/button[4]/i').click() 
            time.sleep(2)
            
            html = BeautifulSoup(driver.page_source, 'html.parser')
            for name2 in html.select('div > div.title'):
                tving_name.append(name2.text)
                tot_list.append(name2.text)
                
            driver.find_element_by_xpath('//*[@id="triggerMenu"]/div/div[1]/div/div/button[4]/i').click()

dist = ['Netflix', 'Tving']
dt = []
netflix_list = {}
tving_list = {}
for i in range(1, 38):
    driver.maximize_window()
    driver.find_element_by_xpath(f'//*[@id="contents"]/div[2]/div/div[{i}]/a/div[2]/div/div/div').click()
    time.sleep(5)
    
    html = BeautifulSoup(driver.page_source, 'html.parser')
    for d in html.select('#editorComment > p'):
        dt.append(d.text)
    dt[i-1] = dt[i-1][0:dt[i-1].index('차')]
    
    if i == 1:
        click_contents(dist[0])
        click_contents(dist[1])
        netflix_list[dt[i-1]] = np.array(netflix_name)
        tving_list[dt[i-1]] = np.array(tving_name)
        time.sleep(3)
        
    elif i >= 2:
        click_contents(dist[0])
        click_contents(dist[1])
        netflix_list[dt[i-1]] = np.array(netflix_name)
        tving_list[dt[i-1]] = np.array(tving_name)
        time.sleep(3)
    
    driver.find_element_by_xpath('//*[@id="header"]/div/button[1]/i').click()
    time.sleep(3)
    print(f'크롤링 {i} 완료. 다음으로 진행 시작.') 

    if i % 11 == 0:
        driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
        time.sleep(2)
        
    if i > 37:
        break  

print('크롤링 끝')

for t in range(36, 0, -1):
    netflix_list[dt[t]] = netflix_list[dt[t]][len(netflix_list[dt[t-1]]):]
    tving_list[dt[t]] = tving_list[dt[t]][len(tving_list[dt[t-1]]):]

In [ ]:
#크롤링 결과 확인
print(netflix_list)
print(tving_list)
print(tot_list)

#중복 제거
print(set(tot_list))
print(len(set(tot_list)))

# 크롤링 결과 데이터 정제하기

## 크롤링 결과 데이터 프레임으로 만들기

In [ ]:
df = pd.DataFrame(columns=['Date', 'Netflix', 'Tving'])

df['Date'] = dt
df['Netflix'] = netflix_list.values()
df['Tving'] = tving_list.values()
df

## 중복제거한 크롤링 컨텐츠명 정제하기

In [ ]:
pd.DataFrame(set(tot_list)).to_csv('result_contents_for_crawling.csv', encoding = 'utf8', index = False)

df3 = pd.read_csv('result_contents_for_crawling.csv', encoding = 'utf8')
df3 = df3['0'].values
df3 = list(df3)
df3.remove('오버로드')
df3.remove('슈퍼 미')
df3.remove('진격의 거인 The Final Season')
df3.remove('일상의 관계')
m_nm = ['아이', '새콤달콤', '극한직업', '노바디', '소울', '1987', '크롤', '미나리', '헌트', '발신제한', '알라딘', '원더 우먼', 
            '인질', '담보', '런', '백두산', '작은 아씨들', '싱크홀', '차인표', '런치박스', '소리도 없이']
d_nm = ['루카', '괴물', 'D.P.', '마우스', '런 온', '로스쿨']

nm_list = []
for m in m_nm: 
    df3 = list(df3)
    df3.remove(m)
    m = f'영화 {m}' 
    nm_list.append(m)
    
for d in d_nm:
    df3 = list(df3)
    df3.remove(d)
    d = f'드라마 {d}' 
    nm_list.append(d)


for i in range(len(df3)):
        if '곡성' in df3[i]:
            df3[i] = '영화 곡성'
        elif '파트' in df3[i]:
            df3[i] = '뤼팽'
        elif '난노' in df3[i]:
            df3[i] = '그녀의 이름은 난노'
        elif '!' in df3[i]:
            df3[i] = df3[i].replace('!', '')
        elif ':' in df3[i]:
            df3[i] = df3[i].replace(':', '')  


print(df3)
print(nm_list)
print(len(df3))

for l in nm_list:
    df3 = list(df3)
    df3.append(l)

for i in range(len(df3)):
    if 'D.P.' in df3[i]:
        df3[i] = '드라마 DP'

pd.DataFrame(df3).to_csv('processed_nm of contents.csv', encoding = 'utf8', index = False)

# SNS 크롤링하기

In [ ]:
driver = webdriver.Chrome('./chromedriver_win32/chromedriver.exe')
url = 'https://www.instagram.com/'
driver.get(url)
time.sleep(4)

ins_id = input('아이디를 입력하세요 : ')
pw = input('비밀번호를 입력하세요 : ')

driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[1]/div/label/input').send_keys(ins_id)
driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[2]/div/label/input').send_keys(pw)
driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[3]/button/div').click()
time.sleep(3)

driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div/div/section/div/button').click()
time.sleep(3)

try:
    driver.find_element_by_xpath('/html/body/div[6]/div/div/div/div[3]/button[2]').click()
    time.sleep(4)

except:
    driver.find_element_by_xpath('/html/body/div[5]/div/div/div/div[3]/button[2]').send_keys('\n')
    time.sleep(4)

key_word = df3
total_cnt = {}
c = 0 
for i, j in zip(key_word, range(len(key_word))):
    c += 1
    print(f'크롤링 중 {c}')
    
    driver.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/input').send_keys('#'+i)
    time.sleep(4)
    
    driver.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/div[3]/div/div[2]/div/div[1]/a/div').click()
    
    html = BeautifulSoup(driver.page_source, 'html.parser')
    try:
        for cnt in html.select('main > header > div.WSpok > div > div:nth-of-type(2) > span > span'):   
            if len(cnt.text.replace(',', "")) >= 4:
                total_cnt[i] = int(cnt.text.replace(',', ""))
                time.sleep(6)

            elif len(cnt.text.replace(',', "")) < 4:
                total_cnt[i] = int(cnt.text) 
                time.sleep(6) 
    
    except:
        for cnt in html.select('main > header > div.WSpok > div > div.qF0y9.Igw0E.IwRSH.eGOV_._4EzTm.a39_R > span > span'):
            if len(cnt.text.replace(',', "")) >= 4:
                total_cnt[i] = int(cnt.text.replace(',', ""))
                time.sleep(6)
            elif len(cnt.text.replace(',', "")) < 4:
                total_cnt[i] = int(cnt.text) 
                time.sleep(6)  
        
    driver.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/input').send_keys(Keys.CONTROL, 'a')
    time.sleep(5)
    driver.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/input').send_keys(Keys.DELETE)
    time.sleep(5)

print(total_cnt)
driver.close()

## SNS 크롤링 결과 데이터 데이터 프레임으로 만들기

In [ ]:
import pandas as pd
k_nm = []
v_nm = []
for k, v in total_cnt.items():
    k_nm.append(k)
    v_nm.append(v)
print(k_nm)
print(v_nm)
df = pd.DataFrame(columns = ['Name', 'Count'])
df['Name'] = k_nm
df['Count'] = v_nm
df

## 크롤링 제대로 안된 부분 다시 크롤링하고 정제하기

In [ ]:
driver = webdriver.Chrome('./chromedriver_win32/chromedriver.exe')
url = 'https://www.instagram.com/'
driver.get(url)
time.sleep(4)

ins_id = input('아이디를 입력하세요 : ')
pw = input('비밀번호를 입력하세요 : ')
driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[1]/div/label/input').send_keys(ins_id)
driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[2]/div/label/input').send_keys(pw)
driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[3]/button/div').click()
time.sleep(3)

driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div/div/section/div/button').click()
time.sleep(3)

try:
    driver.find_element_by_xpath('/html/body/div[6]/div/div/div/div[3]/button[2]').click()
    time.sleep(4)

except:
    driver.find_element_by_xpath('/html/body/div[5]/div/div/div/div[3]/button[2]').send_keys('\n')
    time.sleep(4)
df.to_csv('insta_count.csv', encoding = 'utf8', index = False)
pd.set_option('max_rows', None)
d = pd.read_csv('insta_count.csv', encoding = 'utf8')
a = d[['Name', 'Count']][:95]
b = d['Name'][94:].values
key_word = b
total_cnt1 = {}

c = 0 
for i, j in zip(key_word, range(1, len(key_word))):
    c += 1
    print(f'크롤링 중 {c}')
    
    driver.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/input').send_keys('#'+i)
    time.sleep(4)
    
    driver.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/div[3]/div/div[2]/div/div[1]/a/div').click()
    
    html = BeautifulSoup(driver.page_source, 'html.parser')
    try:
        for cnt in html.select('main > header > div.WSpok > div > div:nth-child(2) > span > span'):  
            if len(cnt.text.replace(',', "")) >= 4:
                total_cnt1[i] = int(cnt.text.replace(',', ""))
                time.sleep(6)
                print(total_cnt1[i])

            elif len(cnt.text.replace(',', "")) < 4:
                total_cnt1[i-1] = int(cnt.text) 
                time.sleep(6)
                print(total_cnt1[i])
    
    except:
        for cnt in html.select('main > header > div.WSpok > div > div.qF0y9.Igw0E.IwRSH.eGOV_._4EzTm.a39_R > span > span'):
            if len(cnt.text.replace(',', "")) >= 4:
                total_cnt1[i] = int(cnt.text.replace(',', ""))
                time.sleep(6)
                print(total_cnt1[i])
            elif len(cnt.text.replace(',', "")) < 4:
                total_cnt1[i] = int(cnt.text) 
                time.sleep(6) 
                print(total_cnt1[i])
        
    driver.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/input').send_keys(Keys.CONTROL, 'a')
    time.sleep(5)
    driver.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/input').send_keys(Keys.DELETE)
    time.sleep(5)

print(total_cnt1)
driver.close()

In [ ]:
del total_cnt1['영화 담보']
import pandas as pd
k1_nm = []
v1_nm = []
for k1, v1 in total_cnt1.items():
    k1_nm.append(k1)
    v1_nm.append(v1)
print(k1_nm)
print(v1_nm)
df1 = pd.DataFrame(columns = ['Name', 'Count'])
df1['Name'] = k1_nm
df1['Count'] = v1_nm
df1

In [ ]:
total = {}
driver.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/input').send_keys('#드라마 로스쿨')
time.sleep(4)

driver.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/div[3]/div/div[2]/div/div[1]/a/div').click()

html = BeautifulSoup(driver.page_source, 'html.parser')
for cnt in html.select('div.WSpok > div > div.qF0y9.Igw0E.IwRSH.eGOV_._4EzTm.a39_R > span > span'):  
    if len(cnt.text.replace(',', "")) >= 4:
        total['드라마 로스쿨'] = int(cnt.text.replace(',', ""))
        time.sleep(6)
        print(total['드라마 로스쿨'])
print(total)

## 완벽하게 된 크롤링 결과를 데이터 프레임으로 만들어주고 컨텐츠명 정제하여 파일로 다시 저장하기

In [ ]:
import pandas as pd
knm = []
vnm = []
for n, m in total.items():
    knm.append(n)
    vnm.append(m)

nm1 = pd.DataFrame(columns = ['Name', 'Count'])
nm1['Name'] = knm
nm1['Count'] = vnm
print(nm1)

new_df = pd.concat([a, df1, nm1], axis = 0)
df_nm = new_df['Name'].values
df_nm = list(df_nm)

print(df_nm)

for i in range(len(df_nm)):
        if '곡성' in df_nm[i]:
            df_nm[i] = '곡성'
        elif '!' in df_nm[i]:
            df_nm[i] = df_nm[i].replace('!', '')
        elif ':' in df_nm[i]:
            df_nm[i] = df_nm[i].replace(':', '')  
        elif 'DP' in df_nm[i]:
            df_nm[i] = 'DP'
        elif ' ' in df_nm[i]:
            df_nm[i]=df_nm[i].replace(" ", "") 
        elif ',' in df_nm[i]:
            df_nm[i] = df_nm[i].replace(',', '')
                   
for j in range(len(df_nm)):
    if '영화' in df_nm[j]:
        df_nm[j] = df_nm[j].strip('영화 ')
    elif '드라' in df_nm[j]:
        df_nm[j] = df_nm[j].strip('드라마 ')
    elif '마우스' in df_nm[j]:
        df_nm[j] = '마우스'
    elif ' ' in df_nm[j]:
            df_nm[j]=df_nm[j].replace(" ", "")
    elif ',' in df_nm[j]:
            df_nm[j] = df_nm[j].replace(',', '')
            
new_df['Name'] = df_nm        
new_df.set_index(['Name'], inplace=True)

print(new_df)

new_df.to_csv('instagram crawling.csv', encoding='utf8')

pd_df = pd.read_csv('instagram crawling.csv', encoding='utf8')

for j in range(len(pd_df)):
    if '언제나그리고영원히' in pd_df['Name'][j]:
            pd_df['Name'][j] = "내가사랑했던모든남자들에게"
    elif '시지프스' in pd_df['Name'][j]:
            pd_df['Name'][j] = "시지프스"        
    elif '우스' in pd_df['Name'][j]:
            pd_df['Name'][j] = '마우스'
pd_df.drop(axis=1, columns = 'Unnamed: 0', inplace=True)
pd_df

In [ ]:
pd_df.to_csv('instagram crawling.csv', encoding='utf8')